In [1]:
import re
import os
import pandas as pd
from termcolor import colored
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
import json

In [2]:
def clean(text,shrink=False):
    text=text.lower()
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text=re.sub('[“"”]',' " ',text)
    retain='[^abcdefghijklmnopqrstuvwxyz!#?" ]'
    text=re.sub('[()–-]',' ',text)
    text=re.sub(retain,'',text)
    text=text.replace('?',' ? ')
    text=text.replace('#',' # ')
    text=text.replace('!',' ! ')
    text=text.split()
    Lemmatizer=WordNetLemmatizer()
    Stemmer=PorterStemmer()
    if shrink:    
        text=[Stemmer.stem(Lemmatizer.lemmatize(word, pos='v')) for word in text]
    return ' '.join(text)

In [3]:
directory='datasets/train-task2-TC.labels'
props_=open(directory).read().split('\n')[:-1]
sentences,Articles, labels=[],[],[]
IDS=[]
Spans=[]
for prop in props_:
    prop_=prop.split('\t')
    x,y=int(prop_[2]),int(prop_[3])
    Spans.append((x,y))
    sentences.append([open('datasets/train-articles/article{}.txt'.format(prop_[0])).read()[:x],open('datasets/train-articles/article{}.txt'.format(prop_[0])).read()[x:y],open('datasets/train-articles/article{}.txt'.format(prop_[0])).read()[y:]])
    Articles.append(open('datasets/train-articles/article{}.txt'.format(prop_[0])).read())
    labels.append(prop_[1])
    IDS.append(prop_[0])
    ids=int(prop_[0])

In [4]:
ind=2
def gen(ind):
    print(labels[ind])
    pre=colored(sentences[ind][0], color='green')
    sen=colored(sentences[ind][1], color='red')
    post=colored(sentences[ind][-1], color='green')
    print(pre+sen+post)
def span_read(id,span):
    return open('datasets/train-articles/article{}.txt'.format(id)).read()[span[0]:span[1]]
def freq_dict(text,Stem=False):
    freq={}
    for word in clean(text,Stem).split():
        if word in freq:
            freq[word]=freq[word]+1
        else:
            freq[word]=1
    return freq

In [6]:
df=pd.DataFrame({'Id':IDS, 'Span':Spans, 'Label':labels})
set(labels)

{'Appeal_to_Authority',
 'Appeal_to_fear-prejudice',
 'Bandwagon,Reductio_ad_hitlerum',
 'Black-and-White_Fallacy',
 'Causal_Oversimplification',
 'Doubt',
 'Exaggeration,Minimisation',
 'Flag-Waving',
 'Loaded_Language',
 'Name_Calling,Labeling',
 'Repetition',
 'Slogans',
 'Thought-terminating_Cliches',
 'Whataboutism,Straw_Men,Red_Herring'}

In [7]:
dumps=True
if(not dumps):
    Stitch=""
    for doc in os.listdir('datasets/train-articles'):
        Stitch=Stitch+open('datasets/train-articles/'+doc).read()    
    f = open("dumps/Text_stitch.txt","w")
    f.write(Stitch)
    f.close()
    corp_freq=dict()
    Doc_freqs=dict()
    corp_freq=freq_dict(Stitch)
    for id in IDS :
        txt=open('datasets/train-articles/article{}.txt'.format(id)).read()
        Doc_freqs[id]=freq_dict(txt)

    corp_freq_json = json.dumps(corp_freq)
    f = open("dumps/Corp_freq_raw.json","w")
    f.write(corp_freq_json)
    f.close()

    Doc_freqs_json = json.dumps(Doc_freqs)
    f = open("dumps/Doc_freqs_raw.json","w")
    f.write(Doc_freqs_json)
    f.close()

    corp_freq_stem=dict()
    Doc_freqs_stem=dict()
    corp_freq_stem=freq_dict(Stitch,Stem=True)
    for id in IDS :
        txt=open('datasets/train-articles/article{}.txt'.format(id)).read()
        Doc_freqs_stem[id]=freq_dict(txt,Stem=True)

    corp_freq_json_stem = json.dumps(corp_freq_stem)
    f = open("dumps/Corp_freq_stem.json","w")
    f.write(corp_freq_json_stem)
    f.close()

    Doc_freqs_json_stem = json.dumps(Doc_freqs_stem)
    f = open("dumps/Doc_freqs_stem.json","w")
    f.write(Doc_freqs_json_stem)
    f.close()

try: 
    Stitch 
except: 
    Stitch=open("dumps/Text_stitch.txt",'r').read()
try: 
    corp_freq_raw 
except: 
    corp_freq_raw=json.loads(open("dumps/Corp_freq_raw.json",'r').read())
try: 
    Doc_freqs_raw 
except: 
    Doc_freqs_raw=json.loads(open("dumps/Doc_freqs_raw.json",'r').read())
try: 
    corp_freq_stem 
except: 
    corp_freq_stem=json.loads(open("dumps/Corp_freq_stem.json",'r').read())
try: 
    Doc_freqs_stem 
except: 
    Doc_freqs_stem=json.loads(open("dumps/Doc_freqs_stem.json",'r').read())

In [21]:
def pad(val):
    t_df=df[df['Label']=='Doubt']
    t_id,t_Span=t_df['Id'],t_df['Span']
    padsent_d=[]
    for ids,spans in zip(t_id,t_Span):
        padsent_d.append(span_read(ids,(spans[0]-val,spans[1]+val)))

    t_df=df[df['Label']!='Doubt']
    t_id,t_Span=t_df['Id'],t_df['Span']
    padsent=[]
    for ids,spans in zip(t_id,t_Span):
        padsent.append(span_read(ids,(spans[0]-val,spans[1]+val)))
    return padsent_d,padsent
padsent_d,padsent=pad(0)

In [17]:
def print_res(val,wts):
    isin,notin=pad(val)
    c,c1,t,t1=0,0,0,0
    box=['?','how','when','where','why','while','what','may','cant']
    for sent in isin:
        t=t+1
        #cond=False
        for check in box:
            if check in clean(sent).split():
                c=c+wts[check]
    for sent in notin:
        t1=t1+1
        #cond=False
        for check in box:
            if check in clean(sent).split():
                c1=c1+wts[check]
    print(c/t,c)
    print(c1/t1,c1)
    print((c*t1)/(c1*t))

In [18]:
wts={
    '?' : 100,
    'how' : 1,
    'when' : 1,
    'where' : 1,
    'why' :1000,
    'while' : 1,
    'what' : 1,
    'may' : 1,
    'cant' : 1,
}
print_res(20,wts)

NameError: name 'padsent_y' is not defined

In [ ]:
clean_stitch=clean(Stitch)
len(clean_stitch)/clean_stitch.count(' ')